## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
gamma = "\u0393"

# Load Data

In [ ]:
## Au ##

ddir = r"E:\atully\k-corrected data\Dec_2020"
file = r"XUV_UPS_2500M0003_kw.h5"  # xuv band structure
# file = r"lamp_overnight_kw.h5"  # lamp band structure

data_au, kx_au, energy_au = loading_functions.load_hdf5(ddir, file)

EF = 20.88  # xuv (hv = 25)
# EF = 16.945  # lamp

energy_au = energy_au - EF

In [ ]:
## 1 ML ##

ddir = r"E:\atully\k-corrected data\Apr_2021\2D_kcorrected"
# file = r"OMBE_XUV_2D0006__kw.h5"  # xuv band structure
file = r"OMBE_Lamp_2D0006__kw.h5"  # lamp band structure

data_1ml, kx_1ml, energy_1ml = loading_functions.load_hdf5(ddir, file)

# EF = 18.27  # xuv (hv = 22.65)
EF = 16.94  # lamp

energy_1ml = energy_1ml - EF

In [ ]:
## 5 ML ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected"  # xuv
# file = r"KE0_kw.h5"  # xuv band structure

ddir = r"E:\atully\arpes_data\2022_June\ARPES\k_corrected"  # lamp
file = r"C60_002_kw.h5"  # lamp band structure

data_5ml, kx_5ml, energy_5ml = loading_functions.load_hdf5(ddir, file)

# EF = 18.48  # xuv (hv = 22.8)
EF = 16.9  # lamp

energy_5ml = energy_5ml - EF

In [ ]:
## 10 ML Lamp ##

ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Swept"  # swept
# file = r"Img0_sweep_kw.h5"
file = r"Img1_avg_g_kw.h5"

data_10ml, kx_10ml, energy_10ml = loading_functions.load_hdf5(ddir, file)

EF = 16.9  # lamp

energy_10ml = energy_10ml - EF

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E - E<sub>F</sub> (eV)"
# yaxis_title = f"E<sub>K</sub> (eV)"
title = f"{file}"

In [ ]:
x, y, data = kx_au, energy_au, data_au
title = "Au(111)"
# x, y, data = kx_1ml, energy_1ml, data_1ml
# title = "1 ML C<sub>60</sub>"
# x, y, data = kx_5ml, energy_5ml, data_5ml
# title = "5 ML C<sub>60</sub>"
# x, y, data = kx_10ml, energy_10ml, data_10ml
# title = "10 ML C<sub>60</sub>"

xlim = (-0.5, 0.47)  # xuv
# ylim = (-4.4, 0)

# xlim = None
ylim = None

# ylim = (-1.2, 0)  # zoom in on mystery feature
# ylim = (-2.0, 0)  # zoom in on mystery feature

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

# Fit for EF

In [ ]:
x, y, data = kx_1ml, energy_1ml, data_1ml

## xuv
xlim = (-0.54, 0.47)
ylim = (20.5, 22)  # EF -- xuv

## lamp
xlim = (-0.5, 0.54)
ylim = (16.6, np.max(y))  # EF -- xuv

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of E<sub>F</sub>",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit FD ##
import lmfit as lm

T = 10  # measurement temp
k_B = 8.617333e-5  # eV/K

x = y_1d
data = col

offset_type = "constant"


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

full_model = lm.models.Model(fermi_dirac) + offset

params = full_model.make_params()

# params["center"].value = 20.88  # xuv
params["center"].value = 16.95  # lamp
params["center"].vary = True
# params["center"].min = 1.85
# params["center"].max = 1.95

params["theta"].value = k_B * T
params["theta"].vary = True


for param in params:
    params[param].value = np.float32(params[param].value)
    params[param].min = np.float32(params[param].min)
    params[param].max = np.float32(params[param].max)
    # print(params[param].value.dtype)


# fit = full_model.fit(data, x=x, params=params, nan_policy="propagate")
fit = full_model.fit(data.astype(np.float32), x=x.astype(np.float32), params=params)

fit.plot()

print(f"EF: {fit.params['center'].value}")

# Plot Data

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    dtick_y=1,
    height=700,
    # height=340,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(colorscale="Blues", reversescale=False, cmin=0, cmax=1)
# fig.update_coloraxes(colorscale="Blues", reversescale=False, cmin=0, cmax=1)
# fig.show(renderer="svg")

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"EDC of C<sub>60</sub> HOMO and HOMO-1",
    # title=f"1ML C<sub>60</sub> EDC (XUV)",
    title=f"EDC",
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=900,
    width=300,
    # dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=col, y=y_1d, name="data", line=dict(color=colors[0])))
fig.update_yaxes(range=(np.min(y_1d), np.max(y_1d)))

fig.show()